In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import gdown
import os,glob
import pandas as pd
from keras.utils import to_categorical
from scipy import sparse
import string
import numpy as np
import json
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
import numpy as np
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku
import tensorflow as tf
tf.random.set_seed(2)
from numpy.random import seed
seed(1)

This code block is used to mount your Google Drive to the Colab environment. It allows you to access files stored in your Google Drive directly from the Colab notebook. The force_remount=True parameter ensures that the drive is remounted if it was previously mounted during the session. This is useful for ensuring that the latest version of the drive's contents is accessible.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


This code block imports essential libraries and modules for data processing, machine learning model building, and text analysis. It includes libraries for data manipulation (Pandas), neural network construction (Keras, TensorFlow), natural language processing (NLTK), and various utilities for handling arrays, strings, and file paths. Additionally, it sets the seed for random number generation to ensure reproducibility.

In [ ]:
import gdown
import os,glob
import pandas as pd
from keras.utils import to_categorical
from scipy import sparse
import string
import numpy as np
import json
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
import numpy as np
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku
import tensorflow as tf
tf.random.set_seed(2)
from numpy.random import seed
seed(1)

In [ ]:
# data_url = "something"

This line of code assigns a URL to the variable data_url. The URL points to a Google Drive folder containing the data files needed for the project. The data can be accessed and downloaded using this URL, typically in combination with Google Drive APIs or tools like gdown to facilitate the download process in a programming environment.

In [ ]:
data_url = "https://drive.google.com/drive/folders/1_RGwmjvW8p3jHuKqIbbHXdBDuDl2DTjU?usp=drive_link"

In [ ]:
gdown.download_folder (data_url, quiet=True, use_cookies=False)

['/content/Guardian News Articles/guardian_articles.csv']

In [ ]:
# Load dataset
data_sample = pd.read_csv('/content/Guardian News Articles/guardian_articles.csv')

# Using only 10% of the data
data = data_sample.sample(frac=0.01, random_state=42)

# Drop rows with missing values
data.dropna(subset=['webTitle', 'bodyContent'], inplace=True)

In [ ]:
data_url = "https://drive.google.com/drive/folders/1_RGwmjvW8p3jHuKqIbbHXdBDuDl2DTjU?usp=drive_link"

This code uses the gdown library to download an entire folder from Google Drive. The data_url specifies the location of the folder on Google Drive. The quiet=True parameter ensures that the download process runs without printing unnecessary logs, and use_cookies=False indicates that cookies should not be used in the download process. This is a convenient way to programmatically download all files from a shared Google Drive folder directly into the runtime environment.

In [ ]:
gdown.download_folder (data_url, quiet=True, use_cookies=False)

['/content/Guardian News Articles/cleaned_guardian_articles_test.csv',
 '/content/Guardian News Articles/cleaned_guardian_articles_train.csv',
 '/content/Guardian News Articles/cleaned_guardian_articles.csv',
 '/content/Guardian News Articles/guardian_articles.csv']

This code block is responsible for loading and cleaning a dataset. It first reads a CSV file named 'guardian_articles.csv' from the specified directory into a pandas DataFrame called data_sample. After loading the data, it removes rows where values in either the 'webTitle' or 'bodyContent' columns are missing. This cleaning step is crucial to ensure the quality and reliability of the dataset for further processing and analysis.

In [ ]:
# # Load dataset
# data = pd.read_csv('/content/Guardian News Articles/guardian_articles.csv')

# # Using only 10% of the data
# # data = data_sample.sample(frac=0.1, random_state=42)

# # Drop rows with missing values
# data.dropna(subset=['webTitle', 'bodyContent'], inplace=True)

This code block sets up the necessary components for advanced text preprocessing. It starts by downloading NLTK datasets for stopwords and WordNet, which are used for text cleaning and normalization. A function clean_text is defined to convert text to lowercase, remove punctuation and numbers, eliminate extra spaces, tokenize, remove stopwords, and apply lemmatization. This function is applied to the 'webTitle' and 'bodyContent' columns of the dataset to clean and preprocess the text. Additionally, special tokens 'starttoken' and 'endtoken' are added to each headline in 'webTitle' to signify the beginning and end, which is a common technique in natural language processing, especially in sequence modeling tasks.

In [ ]:
# Load the cleaned dataset from a CSV file
data = pd.read_csv('/content/Guardian News Articles/cleaned_guardian_articles_train.csv')

# Drop rows with missing values
data.dropna(subset=['webTitle', 'bodyContent','webTitleToken'], inplace=True)

In [ ]:
data

,article_id,sectionName,webTitle,webUrl,bodyContent,webPublicationDate,id,webTitleToken
0,world/2016/jan/31/church-christian-british-adu...,World news,half british adult visited church past year,https://www.theguardian.com/world/2016/jan/31/...,half british adult visited church past year de...,2016-01-31T22:00:09Z,21.0,starttoken half british adult visited church p...
1,tv-and-radio/2016/jan/31/war-and-peace-recap-e...,Television & radio,war peace recap episode five – hero leech cast...,https://www.theguardian.com/tv-and-radio/2016/...,‘strange thing turn sometimes …’ pierre get aw...,2016-01-31T22:00:09Z,22.0,starttoken war peace recap episode five – hero...
2,sport/2016/jan/31/angelique-kerber-serena-will...,Sport,angelique kerber aim dislodge serena williams ...,https://www.theguardian.com/sport/2016/jan/31/...,gone midnight angelique kerber conducting yet ...,2016-01-31T21:59:09Z,23.0,starttoken angelique kerber aim dislodge seren...
3,football/2016/jan/31/fulham-call-off-moussa-de...,Football,fulham call moussa dembéle’s £m move tottenham...,https://www.theguardian.com/football/2016/jan/...,fulham angrily called moussa dembélé’s propose...,2016-01-31T21:56:54Z,24.0,starttoken fulham call moussa dembéle’s £m mov...
4,sport/2016/jan/31/kieran-brookes-england-squad,Sport,northampton kieran brooke drafted england squa...,https://www.theguardian.com/sport/2016/jan/31/...,england drafted northampton prop kieran brooke...,2016-01-31T21:52:14Z,25.0,starttoken northampton kieran brooke drafted e...
...,...,...,...,...,...,...,...,...
148719,world/2022/jun/21/marble-head-of-hercules-pull...,World news,marble head hercules pulled roman shipwreck si...,https://www.theguardian.com/world/2022/jun/21/...,archaeologist it’s underwater find keep giving...,2022-06-21T17:31:32Z,149835.0,starttoken marble head hercules pulled roman s...
148720,music/2022/jun/22/i-got-sick-of-talking-about-...,Music,‘i got sick talking myself’ spacey jane back m...,https://www.theguardian.com/music/2022/jun/22/...,mop curl caleb harper – spacey jane frontman g...,2022-06-21T17:30:09Z,149836.0,starttoken ‘i got sick talking myself’ spacey ...
148721,australia-news/2022/jun/22/the-small-town-with...,Australia news,small town big potato inspired global poetry win,https://www.theguardian.com/australia-news/202...,robertson small pretty town perched edge new s...,2022-06-21T17:30:09Z,149837.0,starttoken small town big potato inspired glob...
148722,australia-news/2022/jun/22/power-to-ban-citize...,Australia news,power ban citizen entering australia questione...,https://www.theguardian.com/australia-news/202...,high court decision striking home affair minis...,2022-06-21T17:30:08Z,149838.0,starttoken power ban citizen entering australi...


In [ ]:
# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Define a function for advanced text cleaning
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove punctuation
    text = re.sub(f"[{string.punctuation}]", " ", text)

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Tokenize text
    tokens = text.split()

    # Remove stopwords
    tokens = [word for word in tokens if word not in stopwords.words('english')]

    # Lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Reconstruct the text from tokens
    text = ' '.join(tokens)

    return text

# Apply advanced text preprocessing
data['webTitle'] = data['webTitle'].apply(clean_text)
data['bodyContent'] = data['bodyContent'].apply(clean_text)

# Add <start> and <end> tokens to each headline
data['webTitle'] = data['webTitle'].apply(lambda x: '<start> ' + x + ' <end>')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Define a function for advanced text cleaning
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove punctuation
    text = re.sub(f"[{string.punctuation}]", " ", text)

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Tokenize text
    tokens = text.split()

    # Remove stopwords
    tokens = [word for word in tokens if word not in stopwords.words('english')]

    # Lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Reconstruct the text from tokens
    text = ' '.join(tokens)

    return text



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def clean_text2(text):
    # Remove specific unwanted characters
    text = text.replace('”', '').replace('–', '')
    return text

In [ ]:
data['webTitle'] = data['webTitle'].apply(clean_text2)
data['bodyContent'] = data['bodyContent'].apply(clean_text2)

In [ ]:
# Initialize the tokenizer without specifying `num_words`
full_tokenizer = Tokenizer()
full_tokenizer.fit_on_texts(data['webTitleToken'])
full_tokenizer.fit_on_texts(data['bodyContent'])

# Get word counts
word_counts = full_tokenizer.word_counts

In [ ]:
data['webTitle']

118646    <start> demand tory mp scrap covid rule could ...
124284    <start> really smart motorway would lower spee...
141061    <start> ‘profiting suffering’ ap cancel sale m...
80327     <start> school prison pipeline criminal justic...
96867     <start> naked coronavirus tale desperation mad...
                                ...                        
140283    <start> uk new zealand sign free trade deal <end>
110667    <start> crisis stage texas border city reel co...
1812      <start> apple iphone sale projected stagnant q...
91415      <start> woke gammon buzzword people coined <end>
31590     <start> arsène wenger say arsenal fa cup win a...
Name: webTitle, Length: 1486, dtype: object

In [ ]:
# Convert the word_counts to a list of (word, count) tuples
word_count_list = [(word, count) for word, count in word_counts.items()]

# Sort the word count list based on counts (descending)
word_count_list.sort(key=lambda x: x[1], reverse=True)

# Print the top 50 words
print("Top 50 words by frequency:", word_count_list[:50])


Top 50 words by frequency: [('said', 486633), ('year', 334501), ('one', 304523), ('would', 271898), ('people', 263636), ('time', 240050), ('say', 212599), ('new', 197682), ('also', 193549), ('like', 184217), ('u', 180507), ('it’s', 169997), ('first', 169517), ('could', 156848), ('two', 153007), ('starttoken', 148695), ('endtoken', 148695), ('day', 144839), ('last', 143214), ('government', 138077), ('make', 128992), ('get', 126799), ('way', 122873), ('work', 121489), ('back', 121474), ('many', 117643), ('world', 113354), ('even', 112328), ('life', 111332), ('week', 109845), ('right', 106036), ('uk', 104378), ('may', 103645), ('made', 103183), ('home', 102982), ('need', 101490), ('much', 99155), ('take', 98164), ('still', 97333), ('go', 97309), ('“i', 95846), ('country', 93366), ('thing', 92898), ('well', 91119), ('woman', 90865), ('three', 90595), ('come', 90131), ('think', 88631), ('don’t', 87589), ('“the', 87567)]


In [ ]:
# Convert the word_counts to a list of (word, count) tuples
word_count_list = [(word, count) for word, count in word_counts.items()]

# Sort the word count list based on counts (descending)
word_count_list.sort(key=lambda x: x[1], reverse=True)

# Print the top 50 words
print("Top 50 words by frequency:", word_count_list[:50])


Top 50 words by frequency: [('said', 486633), ('year', 334501), ('one', 304523), ('would', 271898), ('people', 263636), ('time', 240050), ('say', 212599), ('new', 197682), ('also', 193549), ('like', 184217), ('u', 180507), ('it’s', 169997), ('first', 169517), ('could', 156848), ('two', 153007), ('starttoken', 148695), ('endtoken', 148695), ('day', 144839), ('last', 143214), ('government', 138077), ('make', 128992), ('get', 126799), ('way', 122873), ('work', 121489), ('back', 121474), ('many', 117643), ('world', 113354), ('even', 112328), ('life', 111332), ('week', 109845), ('right', 106036), ('uk', 104378), ('may', 103645), ('made', 103183), ('home', 102982), ('need', 101490), ('much', 99155), ('take', 98164), ('still', 97333), ('go', 97309), ('“i', 95846), ('country', 93366), ('thing', 92898), ('well', 91119), ('woman', 90865), ('three', 90595), ('come', 90131), ('think', 88631), ('don’t', 87589), ('“the', 87567)]


In [ ]:
# Calculate cumulative word frequency
cumulative_frequency = np.cumsum([count for word, count in word_count_list])

# Total number of words
total_words = cumulative_frequency[-1]

# Determine a threshold (e.g., 95% of the total frequency)
threshold = total_words * 0.95

# Find the vocabulary size needed to reach this threshold
vocab_size = next(i for i, total in enumerate(cumulative_frequency) if total > threshold)

print("Suggested vocabulary size:", vocab_size)

Suggested vocabulary size: 29177


In [ ]:
data['webTitleToken']

0         starttoken half british adult visited church p...
1         starttoken war peace recap episode five – hero...
2         starttoken angelique kerber aim dislodge seren...
3         starttoken fulham call moussa dembéle’s £m mov...
4         starttoken northampton kieran brooke drafted e...
                                ...                        
148719    starttoken marble head hercules pulled roman s...
148720    starttoken ‘i got sick talking myself’ spacey ...
148721    starttoken small town big potato inspired glob...
148722    starttoken power ban citizen entering australi...
148723    starttoken liberal mp say peter dutton let par...
Name: webTitleToken, Length: 148695, dtype: object

In [ ]:
# Initialize tokenizer with a smaller vocabulary
max_vocab_size = 50  # Adjust as needed
tokenizer = Tokenizer(num_words=max_vocab_size)
tokenizer.fit_on_texts(data['webTitle'])
tokenizer.fit_on_texts(data['bodyContent'])

# Convert texts to sequences
sequences_body = tokenizer.texts_to_sequences(data['bodyContent'])
sequences_title = tokenizer.texts_to_sequences(data['webTitle'])

# Padding sequences
body_padded = pad_sequences(sequences_body, maxlen=300)  # Adjust maxlen as per your data
title_padded = pad_sequences(sequences_title, maxlen=30, padding='post')  # Adjust maxlen as per your data

# Preparing decoder input
title_padded_shifted = np.zeros_like(title_padded)
title_padded_shifted[:, 1:] = title_padded[:, :-1]

In [ ]:
# Convert the word_counts to a list of (word, count) tuples
word_count_list = [(word, count) for word, count in word_counts.items()]

# Sort the word count list based on counts (descending)
word_count_list.sort(key=lambda x: x[1], reverse=True)

# Print the top 50 words
print("Top 50 words by frequency:", word_count_list[:50])


Top 50 words by frequency: [('said', 486633), ('year', 334501), ('one', 304523), ('would', 271898), ('people', 263636), ('time', 240050), ('say', 212599), ('new', 197682), ('also', 193549), ('like', 184217), ('u', 180507), ('it’s', 169997), ('first', 169517), ('could', 156848), ('two', 153007), ('starttoken', 148695), ('endtoken', 148695), ('day', 144839), ('last', 143214), ('government', 138077), ('make', 128992), ('get', 126799), ('way', 122873), ('work', 121489), ('back', 121474), ('many', 117643), ('world', 113354), ('even', 112328), ('life', 111332), ('week', 109845), ('right', 106036), ('uk', 104378), ('may', 103645), ('made', 103183), ('home', 102982), ('need', 101490), ('much', 99155), ('take', 98164), ('still', 97333), ('go', 97309), ('“i', 95846), ('country', 93366), ('thing', 92898), ('well', 91119), ('woman', 90865), ('three', 90595), ('come', 90131), ('think', 88631), ('don’t', 87589), ('“the', 87567)]


In [ ]:
# Convert the word_counts to a list of (word, count) tuples
word_count_list = [(word, count) for word, count in word_counts.items()]

# Sort the word count list based on counts (descending)
word_count_list.sort(key=lambda x: x[1], reverse=True)

# Print the top 50 words
print("Top 50 words by frequency:", word_count_list[:50])


Top 50 words by frequency: [('said', 486633), ('year', 334501), ('one', 304523), ('would', 271898), ('people', 263636), ('time', 240050), ('say', 212599), ('new', 197682), ('also', 193549), ('like', 184217), ('u', 180507), ('it’s', 169997), ('first', 169517), ('could', 156848), ('two', 153007), ('starttoken', 148695), ('endtoken', 148695), ('day', 144839), ('last', 143214), ('government', 138077), ('make', 128992), ('get', 126799), ('way', 122873), ('work', 121489), ('back', 121474), ('many', 117643), ('world', 113354), ('even', 112328), ('life', 111332), ('week', 109845), ('right', 106036), ('uk', 104378), ('may', 103645), ('made', 103183), ('home', 102982), ('need', 101490), ('much', 99155), ('take', 98164), ('still', 97333), ('go', 97309), ('“i', 95846), ('country', 93366), ('thing', 92898), ('well', 91119), ('woman', 90865), ('three', 90595), ('come', 90131), ('think', 88631), ('don’t', 87589), ('“the', 87567)]


In [ ]:
# Calculate cumulative word frequency
cumulative_frequency = np.cumsum([count for word, count in word_count_list])

# Total number of words
total_words = cumulative_frequency[-1]

# Determine a threshold (e.g., 95% of the total frequency)
threshold = total_words * 0.95

# Find the vocabulary size needed to reach this threshold
vocab_size = next(i for i, total in enumerate(cumulative_frequency) if total > threshold)

print("Suggested vocabulary size:", vocab_size)

Suggested vocabulary size: 29177


In [ ]:
data['webTitleToken']

0         starttoken half british adult visited church p...
1         starttoken war peace recap episode five – hero...
2         starttoken angelique kerber aim dislodge seren...
3         starttoken fulham call moussa dembéle’s £m mov...
4         starttoken northampton kieran brooke drafted e...
                                ...                        
148719    starttoken marble head hercules pulled roman s...
148720    starttoken ‘i got sick talking myself’ spacey ...
148721    starttoken small town big potato inspired glob...
148722    starttoken power ban citizen entering australi...
148723    starttoken liberal mp say peter dutton let par...
Name: webTitleToken, Length: 148695, dtype: object

This code initializes a tokenizer with a specified maximum vocabulary size and fits it on the text data. It creates a Tokenizer object with a vocabulary limit (max_vocab_size) set to 50. The tokenizer is then trained on the 'webTitle' and 'bodyContent' text data from the dataset. After fitting, it checks if the special token 'starttoken' is part of the tokenizer's word index. This is essential for sequence-to-sequence models in NLP, as it indicates whether the special tokens used to denote the start of sequences are recognized by the tokenizer. The output will confirm if 'starttoken' is included in the tokenizer's vocabulary.

In [ ]:
# Initialize tokenizer with a smaller vocabulary
max_vocab_size = 30000  # Adjust as needed
tokenizer = Tokenizer(num_words=max_vocab_size)
tokenizer.fit_on_texts(data['webTitleToken'])
tokenizer.fit_on_texts(data['bodyContent'])

# Check if 'starttoken' token is in the tokenizer's word index
if 'starttoken' in tokenizer.word_index:
    print("'starttoken' token is in the tokenizer's word index.")
else:
    print("'starttoken' token is NOT in the tokenizer's word index.")


'starttoken' token is in the tokenizer's word index.


This code block converts text data into sequences of integers and pads them for consistent length, preparing them for input into a neural network. It first transforms the text in 'bodyContent' and 'webTitle' columns into sequences of integers using the previously trained tokenizer. Then, it pads these sequences to ensure they all have the same length: 300 for the body content and 30 for the titles. This uniform length is necessary for batch processing in neural networks. Additionally, a shifted version of the title sequences is created, which is commonly used as input to the decoder in sequence-to-sequence models, facilitating the prediction of the next word in the sequence.

In [ ]:
# Convert texts to sequences
sequences_body = tokenizer.texts_to_sequences(data['bodyContent'])
sequences_title = tokenizer.texts_to_sequences(data['webTitleToken'])

# Padding sequences
body_padded = pad_sequences(sequences_body, maxlen=500)  # Adjust maxlen as per your data
title_padded = pad_sequences(sequences_title, maxlen=30, padding='post')  # Adjust maxlen as per your data

# Preparing decoder input
title_padded_shifted = np.zeros_like(title_padded)
title_padded_shifted[:, 1:] = title_padded[:, :-1]

This code block saves the trained tokenizer to a file for future use. It utilizes the pickle module to serialize the tokenizer object and writes it to a file named 'tokenizer.pickle' in a specified directory on Google Drive. The pickle.HIGHEST_PROTOCOL is used for efficient serialization. Saving the tokenizer is important because it ensures consistency in tokenization when preprocessing new data for predictions, especially if the model is to be used or deployed later.

In [ ]:
from keras.models import Sequential, load_model
import pickle
with open('/content/drive/MyDrive/path/Models/seq2seq_headline_generator/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Splitting data into training and validation sets
X_train, X_val, title_train, title_val, decoder_input_train, decoder_input_val = train_test_split(
    body_padded, title_padded, title_padded_shifted, test_size=0.2, random_state=42)

In [ ]:
# Function to convert to sparse categorical
def to_sparse_categorical(y, num_classes):
    y_sparse = []
    for sequence in y:
        y_sparse_seq = np.zeros((len(sequence), num_classes))
        for i, idx in enumerate(sequence):
            if idx < num_classes:
                y_sparse_seq[i, idx] = 1
        y_sparse.append(y_sparse_seq)
    return np.array(y_sparse)

# Converting training and validation target data to sparse categorical
num_classes = max_vocab_size + 1  # +1 for the padding token
y_train_sparse = to_sparse_categorical(title_train, num_classes)
y_val_sparse = to_sparse_categorical(title_val, num_classes)

This code block is responsible for splitting the padded sequences into training and validation sets. It uses the train_test_split function from the sklearn.model_selection module to divide the dataset into two parts: one for training the model (X_train, title_train, decoder_input_train) and the other for validating the model's performance (X_val, title_val, decoder_input_val). The test_size=0.2 parameter specifies that 20% of the data should be reserved for the validation set, while the remaining 80% is used for training. The random_state=42 ensures that the split is reproducible, meaning the same split will occur each time the code is run.

In [ ]:
# Splitting data into training and validation sets
X_train, X_val, title_train, title_val, decoder_input_train, decoder_input_val = train_test_split(
    body_padded, title_padded, title_padded_shifted, test_size=0.2, random_state=42)

This code block includes a function to convert sequence data into sparse categorical format and applies this conversion to the training and validation sets. The function to_sparse_categorical takes each sequence (y) and the number of classes (num_classes) as inputs. It creates a sparse matrix where each row corresponds to a sequence, and each column represents a class (word in the vocabulary). For each word index in the sequence, it sets the corresponding column to 1, resulting in a one-hot encoded representation of the sequences. This sparse format is particularly useful for handling large vocabularies in classification tasks. The function is then used to convert both the training (title_train) and validation (title_val) target data into this sparse categorical format, facilitating their use in training a neural network model for sequence prediction tasks.

In [ ]:
max_features = max_vocab_size + 1  # +1 for padding token
embedding_dim = 128

In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding, Concatenate, Bidirectional

# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(input_dim=max_vocab_size + 1, output_dim=128)
encoder_lstm = Bidirectional(LSTM(128, return_state=True))
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_lstm(encoder_embedding(encoder_inputs))
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding_layer = Embedding(input_dim=max_vocab_size + 1, output_dim=128)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_dense = Dense(max_vocab_size + 1, activation='softmax')  # Correct number of units

decoder_embedding = decoder_embedding_layer(decoder_inputs)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_outputs = decoder_dense(decoder_outputs)

# Seq2Seq Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy')


In [ ]:
model.summary()

Model Overview:

This code sets up a sequence-to-sequence (Seq2Seq) neural network model, commonly used in natural language processing tasks like machine translation or text summarization.
The model consists of two main components: an encoder and a decoder, both utilizing Long Short-Term Memory (LSTM) networks, which are effective in handling sequential data.
Encoder:

Input Layer: encoder_inputs is the input layer, accepting sequences of indefinite length.
Embedding Layer: Transforms the input sequences into dense vectors of fixed size (128 dimensions in this case).
Dropout Layer: Applied after the embedding layer to prevent overfitting by randomly setting a fraction of the input units to 0.
Stacked LSTM Layers: Two bidirectional LSTM layers (encoder_lstm and encoder_lstm_2) are used. The first LSTM layer returns sequences, while the second returns only the final states. These layers help the model to capture information from both directions of the input sequence (forward and backward).
States Concatenation: The final hidden and cell states (state_h and state_c) are concatenated to form the complete state representation from both LSTMs.
Decoder:

Input Layer: decoder_inputs is the input layer for the decoder.
Embedding Layer: Similar to the encoder, it converts the decoder input to dense vectors.
Dropout Layer: Applied to the decoder's embedding output.
LSTM Layer: A single LSTM layer with a higher number of units (1024) processes the input sequence, utilizing the encoder's final states as its initial states.
Dense Layers: Two Dense layers are added. The first (decoder_dense_1) with 512 units and ReLU activation introduces non-linearity and complexity. The second (decoder_dense_2) with softmax activation outputs a probability distribution over the vocabulary for each output sequence element.
Seq2Seq Model Construction:

The model is constructed by connecting the encoder and decoder components.
The encoder's output states are fed into the decoder LSTM as initial states, facilitating the flow of information from the encoder to the decoder.
Compilation:

The model is compiled with the Adam optimizer and categorical cross-entropy loss, which is standard for multi-class classification problems like this.
Key Aspects:

The use of bidirectional LSTMs in the encoder enables the model to capture context from both directions, improving its understanding of the input sequence.
The increased complexity in the decoder, with additional LSTM units and Dense layers, allows the model to generate more nuanced and accurate output sequences.
Dropout layers aid in reducing overfitting, especially crucial in a model with high complexity and many parameters.

In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding, Concatenate, Bidirectional, Dropout

# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(input_dim=max_vocab_size + 1, output_dim=128)(encoder_inputs)
encoder_dropout = Dropout(0.5)(encoder_embedding)  # Dropout layer after embedding
# Stacking LSTMs in the encoder
encoder_lstm = Bidirectional(LSTM(512, return_sequences=True, return_state=True))
encoder_lstm_2 = Bidirectional(LSTM(512, return_state=True))

# Applying the first LSTM and then the second
encoder_output, forward_h, forward_c, backward_h, backward_c = encoder_lstm(encoder_dropout)
_, forward_h2, forward_c2, backward_h2, backward_c2 = encoder_lstm_2(encoder_output)

# Using the states from the last LSTM layer
state_h = Concatenate()([forward_h2, backward_h2])
state_c = Concatenate()([forward_c2, backward_c2])
encoder_states = [state_h, state_c]

# Adjust the size of the states to match the decoder LSTM's expected state size
state_h = Dense(1024, activation='relu')(state_h)  # Reduce size to 1024
state_c = Dense(1024, activation='relu')(state_c)  # Reduce size to 1024
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding_layer = Embedding(input_dim=max_vocab_size + 1, output_dim=128)
decoder_embedding = decoder_embedding_layer(decoder_inputs)
decoder_dropout = Dropout(0.5)(decoder_embedding)  # Dropout layer after embedding
decoder_lstm = LSTM(1024, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_dropout, initial_state=encoder_states)

# Adding the new Dense layers
decoder_dense_1 = Dense(1024, activation='relu')  # New intermediate Dense layer
decoder_intermediate = decoder_dense_1(decoder_outputs)  # Apply the new Dense layer
decoder_dense_2 = Dense(max_vocab_size + 1, activation='softmax')  # Final Dense layer for output
decoder_outputs = decoder_dense_2(decoder_intermediate)  # Apply the final Dense layer

# Seq2Seq Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 128)            3840128   ['input_1[0][0]']             
                                                                                                  
 dropout (Dropout)           (None, None, 128)            0         ['embedding[0][0]']           
                                                                                                  
 bidirectional (Bidirection  [(None, None, 1024),         2625536   ['dropout[0][0]']             
 al)                          (None, 512),                                                    

In [ ]:
history = model.fit([X_train, decoder_input_train], y_train_sparse,
                    validation_data=([X_val, decoder_input_val], y_val_sparse),
                    epochs=5,  # Adjust as needed
                    batch_size=32)

In [ ]:
from keras.models import Sequential, load_model
import pickle
#saving model
model.save('/content/drive/MyDrive/path/Models/seq2seq_headline_generator/seq2seq_headline_generator.h5')
with open('/content/drive/My Drive/path/Models/seq2seq_headline_generator/history.p', 'wb') as f:
    pickle.dump(history.history, f)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


This code block is for training the Seq2Seq model with early stopping. It trains the model on the training data and validates it on the validation data, stopping the training early if there's no improvement in validation loss for two consecutive epochs. This approach helps in preventing overfitting and ensures efficient training. The model is trained for a maximum of 10 epochs with a batch size of 64, and the training progress is monitored using the EarlyStopping callback.

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=2)

# Train the model
history = model.fit([X_train, decoder_input_train], title_train,
                    validation_data=([X_val, decoder_input_val], title_val),
                    epochs=50,  # Adjust epochs as needed
                    batch_size=128)  # Adjust batch size as needed

Epoch 1/50
930/930 [==============================] - 745s 801ms/step - loss: 2.2019 - val_loss: 2.1414
Epoch 2/50
930/930 [==============================] - 718s 772ms/step - loss: 2.0811 - val_loss: 2.0564
Epoch 3/50
930/930 [==============================] - 710s 764ms/step - loss: 1.9749 - val_loss: 1.9826
Epoch 4/50
930/930 [==============================] - 706s 760ms/step - loss: 1.8827 - val_loss: 1.9502
Epoch 5/50
930/930 [==============================] - 702s 754ms/step - loss: 1.8054 - val_loss: 1.9416
Epoch 6/50
930/930 [==============================] - 701s 754ms/step - loss: 1.7327 - val_loss: 1.9506
Epoch 7/50
930/930 [==============================] - 699s 752ms/step - loss: 1.6568 - val_loss: 1.9829
Epoch 8/50
930/930 [==============================] - 700s 753ms/step - loss: 1.5830 - val_loss: 2.0192
Epoch 9/50
930/930 [==============================] - 698s 750ms/step - loss: 1.5137 - val_loss: 2.0799
Epoch 10/50
930/930 [==============================] - 698s 751m

This code block saves the trained Seq2Seq model and its training history. The model is saved as an H5 file (seq2seq_headline_generator.h5) to a specified path on Google Drive. Additionally, the training history, which includes metrics like loss and accuracy for each epoch, is saved as a pickle file (history.p). This allows for later analysis of the model's training process and easy reloading of the model for further use or evaluation.

In [ ]:
from keras.models import Sequential, load_model
import pickle
#saving model
model.save('/content/drive/MyDrive/path/Models/seq2seq_headline_generator/seq2seq_headline_generator.h5')
with open('/content/drive/My Drive/path/Models/seq2seq_headline_generator/history.p', 'wb') as f:
    pickle.dump(history.history, f)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


This code block is for loading the previously trained Seq2Seq model and the associated tokenizer. The load_model function from Keras is used to load the saved model file (seq2seq_headline_generator.h5) from Google Drive. Additionally, the tokenizer, which is essential for processing text inputs in the same format as the training data, is loaded from a saved pickle file (tokenizer.pickle) using the pickle module. This setup is crucial for making new predictions or continuing the model's evaluation using the same tokenization scheme as used during training.

In [ ]:
from keras.models import load_model, Model
import numpy as np

# Load the trained model
model = load_model('/content/drive/MyDrive/path/Models/seq2seq_headline_generator/seq2seq_headline_generator.h5')

# Load the tokenizer
with open('/content/drive/MyDrive/path/Models/seq2seq_headline_generator/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
from keras.models import Sequential, load_model
import pickle
#saving model
model.save('/content/drive/MyDrive/path/Models/seq2seq_headline_generator/seq2seq_headline_generator.h5')
with open('/content/drive/My Drive/path/Models/seq2seq_headline_generator/history.p', 'wb') as f:
    pickle.dump(history.history, f)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


This code block is for loading the previously trained Seq2Seq model and the associated tokenizer. The load_model function from Keras is used to load the saved model file (seq2seq_headline_generator.h5) from Google Drive. Additionally, the tokenizer, which is essential for processing text inputs in the same format as the training data, is loaded from a saved pickle file (tokenizer.pickle) using the pickle module. This setup is crucial for making new predictions or continuing the model's evaluation using the same tokenization scheme as used during training.

In [ ]:
from keras.models import load_model, Model
import numpy as np

# Load the trained model
model = load_model('/content/drive/MyDrive/path/Models/seq2seq_headline_generator/seq2seq_headline_generator.h5')

# Load the tokenizer
with open('/content/drive/MyDrive/path/Models/seq2seq_headline_generator/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

This code block creates separate models for the encoder and decoder parts of the previously trained Seq2Seq model, which is useful for making predictions. The encoder model is defined to take the original encoder inputs and output the encoder states. For the decoder model, new input layers are created to receive the hidden and cell states, and it is set up to take these states along with the decoder inputs. The decoder model then outputs the predicted sequences and the updated states. This separation into encoder and decoder models is particularly useful for generating sequences in an iterative manner, where the state outputs from one time step are fed back as inputs for the next time step.

In [ ]:
# Encoder Model
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

# Inputs for the decoder model
decoder_state_input_h = Input(shape=(1024,))
decoder_state_input_c = Input(shape=(1024,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_inputs_single = Input(shape=(1,))  # Single timestep input

# Decoder setup
decoder_state_input_h = Input(shape=(1024,))  # Keep the state size as 1024
decoder_state_input_c = Input(shape=(1024,))  # Keep the state size as 1024
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_embedding_single = decoder_embedding_layer(decoder_inputs_single)
decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding_single, initial_state=decoder_states_inputs)

decoder_outputs = decoder_dense_2(decoder_outputs)

# Final decoder model
decoder_model = Model(
    [decoder_inputs_single] + decoder_states_inputs,
    [decoder_outputs, state_h, state_c]
)


In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors
    states_value = encoder_model.predict(input_seq)

    # Start with a sequence containing just the start token for the decoder
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tokenizer.word_index['<start>']  # Assuming '<start>' is in the tokenizer


    # Create the translation
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token and add the corresponding word to the decoded sentence
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tokenizer.index_word[sampled_token_index]
        decoded_sentence += ' ' + sampled_word

        # Exit condition: either hit max length or find the stop token
        if (sampled_word == '<end>' or len(decoded_sentence) > 50):  # Adjust max length as needed
            stop_condition = True

        # Update the target sequence (length 1)
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [ ]:
input_seq = X_train[0:1]  # Take one sequence from the training set
input_seq

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  592,
           5, 3871,  869,    3, 4074, 1042,  162,  596,  406, 2993, 3871,
          19,  748,  152,  817, 1769, 

In [ ]:
if '<start>' in tokenizer.word_index:
    print("'<start>' token is in the tokenizer's word index.")
else:
    print("'<start>' token is NOT in the tokenizer's word index.")

'<start>' token is NOT in the tokenizer's word index.


In [ ]:
print(list(tokenizer.word_index.items())[:50])  # Print first 50 entries of the tokenizer's word index

[('”', 1), ('–', 2), ('said', 3), ('year', 4), ('one', 5), ('would', 6), ('people', 7), ('time', 8), ('end', 9), ('say', 10), ('new', 11), ('also', 12), ('start', 13), ('like', 14), ('u', 15), ('first', 16), ('it’s', 17), ('could', 18), ('two', 19), ('day', 20), ('last', 21), ('government', 22), ('make', 23), ('get', 24), ('way', 25), ('work', 26), ('back', 27), ('many', 28), ('even', 29), ('week', 30), ('world', 31), ('life', 32), ('uk', 33), ('may', 34), ('home', 35), ('right', 36), ('need', 37), ('made', 38), ('much', 39), ('take', 40), ('go', 41), ('“i', 42), ('still', 43), ('country', 44), ('thing', 45), ('well', 46), ('woman', 47), ('three', 48), ('think', 49), ('don’t', 50)]


In [ ]:
generated_headline = decode_sequence(input_seq)
print(generated_headline)

1/1 [==============================] - 0s 27ms/step


KeyError: ignored